In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from transformers import RobertaTokenizer, RobertaConfig, RobertaModel
from collections.abc import Iterable
from transformers import AutoTokenizer, AutoModel
import torch
from collections import deque
from tqdm import tqdm
import pickle
import re
from itertools import product

/Users/cslab/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


임베딩 만드는 소스 코드 (챗지피티 전후로의 임베딩)

In [2]:
# 파일 불러오기
with open('../../data/bert_src_df.pkl', 'rb') as f:
    bert_src_df = pickle.load(f)


In [3]:
bert_src_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5227010 entries, 0 to 5227009
Data columns (total 5 columns):
 #   Column          Dtype         
---  ------          -----         
 0   q_id            int64         
 1   a_id            float64       
 2   q_creationdate  datetime64[ns]
 3   tags            object        
 4   body            object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 199.4+ MB


In [4]:
cond1 = bert_src_df['q_creationdate']<='2022-11-30'
cond2 = bert_src_df['q_creationdate']>='2021-11-30'
cond3 = bert_src_df['a_id'].isna()
cond4 = bert_src_df['tags'].str.contains('python')
cond5 = bert_src_df['q_creationdate']>='2022-12-01'

In [5]:
bert_src = bert_src_df.loc[cond2 & cond4, :]

In [6]:
bert_src.head()

,q_id,a_id,q_creationdate,tags,body
1,76125232,NaN,2023-04-27 23:10:20.003,<python><python-3.x><django><django-rest-frame...,<p>Perfoming a crud api rest in django rest fr...
3,72041563,72041734.0,2022-04-28 09:50:22.723,<python><pandas><dataframe>,<p>I want to replace some strings in a column ...
4,70538493,70637062.0,2021-12-31 02:31:59.197,<python><selenium><google-chrome><selenium-chr...,<p>I have a selenium script that I execute in ...
5,70538493,70642594.0,2021-12-31 02:31:59.197,<python><selenium><google-chrome><selenium-chr...,<p>I have a selenium script that I execute in ...
26,76917614,76918200.0,2023-08-16 23:26:19.093,<python><list>,<p>My list consists on several items:</p>\n<pr...


In [7]:
def cleanhtml(raw_html):
  # 1.Source code in python language is hard to understand, so replace all the <code> tag first
  cleantext_1 = re.findall(r'(?<=\<code>)(.*?)(?=<\/code>)', raw_html.replace('\n', '_**_'))
  cleantext_1 = [x.replace('_**_', '\n') for x in cleantext_1]
  # 2. replace html tags
  # <p>
  tag_re = re.compile('<.*?>')
  cleantext_2 = [re.sub(tag_re, '', x) for x in cleantext_1]
  return cleantext_2

In [8]:
# apply the function, cleanhtml to the question and body text
bert_src.loc[:, 'q_prep_text'] = bert_src['body'].apply(cleanhtml)

/var/folders/lk/t243gzpx379bzd9mvx8dcsmm0000gp/T/ipykernel_17783/291002259.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bert_src.loc[:, 'q_prep_text'] = bert_src['body'].apply(cleanhtml)


In [9]:
bert_src = bert_src.reset_index(drop=True)

In [10]:
bert_src_ex = bert_src[['q_id', 'q_prep_text']].apply(pd.Series.explode)

In [11]:
bert_src_ex.dropna(inplace=True)

In [12]:
bert_src = pd.merge(bert_src_ex, bert_src[['q_id', 'q_creationdate']], how ='left', left_on = 'q_id', right_on = 'q_id')

In [13]:
bert_src['q_year'] = pd.to_datetime(bert_src['q_creationdate']).dt.year
bert_src['q_month'] = pd.to_datetime(bert_src['q_creationdate']).dt.month

In [14]:
# PREPROCESSING FOR CODE SCRIPT
def preprocess_script(script):
    new_script = deque()
    old_script = script.split('\n')
    for line in old_script:
        if line.lstrip().startswith('#'): # 주석으로 시작되는 행 skip
            continue
        line = line.rstrip()
        if '#' in line:
            line = line[:line.index('#')] # 주석 전까지 코드만 저장
        line = line.replace('\n','') # 개행 문자를 모두 삭제함
        line = line.replace('    ','\t') # 공백 4칸을 tab으로 변환
        
        if line == '': # 전처리 후 빈 라인은 skip
            continue
        
        new_script.append(line)

        
    new_script = '\n'.join(new_script) # 개행 문자로 합침
    new_script = re.sub('("""[\w\W]*?""")', '<str>', new_script)
    new_script = re.sub("('''[\w\W]*?''')", '<str>', new_script)
    new_script = re.sub('/^(http?|https?):\/\/([a-z0-9-]+\.)+[a-z0-9]{2,4}.*$/', '', new_script)
    
    return new_script


In [15]:
bert_src['q_prep_text_non'] = bert_src['q_prep_text'].apply(preprocess_script)

In [16]:
years = [2022, 2023]
months = [x for x in range(1, 13)]

In [17]:
ym = list(product(years, months))

In [18]:
# CodeBERT 모델과 토크나이저 로드
model_name = "microsoft/codebert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

In [19]:
# 임베딩 함수 정의
def embed_text(text):
    encoded_input = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**encoded_input)
    return model_output.last_hidden_state[:, 0, :]

In [23]:
ym[10:12]

[(2022, 11), (2022, 12)]

In [20]:
for year, month in ym[10:12] : 
    cond1 = (bert_src['q_year'] ==year)
    cond2 = (bert_src['q_month'] ==month)
    src = bert_src.loc[cond1&cond2, 'q_prep_text_non'].tolist()

    print(year, month)
    embed_text_list = []
    for i in tqdm(range(len(src))) : 
        embed_text_list.append(embed_text(src[i]))

    file_nm = 'embed_text_list'+str(year)+str(month)+'.pkl'
    # 전체 데이터셋 
    with open('../../data/src/embeding/'+file_nm, 'wb') as f:
        pickle.dump(embed_text_list, f)

 35%|███▌      | 50739/144351 [41:17<1:16:10, 20.48it/s]


KeyboardInterrupt: 